In [1]:
# # the imports in this cell are required when running on local device
# import os, sys
# sys.path.append(os.path.join('..', '..'))
# from utils.applyML_util import train_regression, eval_regression
# from utils.featureSelection_util import (pearson_correlation_fs, 
#                                          seleckKBest_fs, selectSequential_fs)

In [2]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files 
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# New Section

**GNB Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

**XGBoost Documentation link:** https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html#Zero-Inflated-Regressor

In [3]:
!pip install sklego

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklego.meta import ZeroInflatedRegressor #!pip install sklego
from xgboost.sklearn import XGBRegressor
from sklearn.naive_bayes import GaussianNB

In [5]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=GaussianNB(), regressor=XGBRegressor(random_state=RAND_SEED, n_jobs=-1, objective ='reg:squarederror'))

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
classifier__var_smoothing = [1e-14,1e-13, 1e-12,1e-11, 1e-10, 1e-9, 1e-8, 1e-7]


regressor__n_estimators = [1, 10, 50, 100, 150]
regressor__early_stopping_rounds = [5]
regressor__learning_rate = [ 0.01, 0.1, 1.0]
regressor__gamma = [0, 1 , 5, 10]
regressor__subsample = [ 0.5, 1.0]


param_grid = {'classifier__var_smoothing' : classifier__var_smoothing,
              'regressor__n_estimators':regressor__n_estimators, 'regressor__learning_rate':regressor__learning_rate,
              'regressor__gamma': regressor__gamma, 'regressor__subsample': regressor__subsample, 'regressor__early_stopping_rounds': regressor__early_stopping_rounds}
# variables needed for showEvalGraph_regression() function
# MODEL_CLASS = ziReg
# x_axis_param_name = 'regressor__n_estimators'
# x_axis_vals = n_estimators

In [6]:
ziReg.get_params().keys()


dict_keys(['classifier__priors', 'classifier__var_smoothing', 'classifier', 'regressor__base_score', 'regressor__booster', 'regressor__colsample_bylevel', 'regressor__colsample_bynode', 'regressor__colsample_bytree', 'regressor__gamma', 'regressor__importance_type', 'regressor__learning_rate', 'regressor__max_delta_step', 'regressor__max_depth', 'regressor__min_child_weight', 'regressor__missing', 'regressor__n_estimators', 'regressor__n_jobs', 'regressor__nthread', 'regressor__objective', 'regressor__random_state', 'regressor__reg_alpha', 'regressor__reg_lambda', 'regressor__scale_pos_weight', 'regressor__seed', 'regressor__silent', 'regressor__subsample', 'regressor__verbosity', 'regressor'])

## 1. Experimentation on the Weather Daily dataset

In [7]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [8]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 50, 'regressor__subsample': 1.0}
Train set performance: r2-score=0.2321, mae=5.6068, rmse=13.0484


In [9]:
# # r2-scores graph on the train set
# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [10]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.15, mae=5.9979, rmse=15.9417


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [11]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [12]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 10, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 50, 'regressor__subsample': 1.0}
Train set performance: r2-score=0.2329, mae=5.6877, rmse=13.0291


In [13]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [14]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.147, mae=6.1231, rmse=15.9693


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [15]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Relative Humidity (morning, %)']


In [16]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 50, 'regressor__subsample': 0.5}
Train set performance: r2-score=0.2287, mae=5.6199, rmse=13.078


In [17]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [18]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1137, mae=6.0709, rmse=16.2787


### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [19]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Actual Evaporation (mm)', 'Relative Humidity (morning, %)']


In [20]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 10, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 50, 'regressor__subsample': 1.0}
Train set performance: r2-score=0.2134, mae=5.6961, rmse=13.2118


In [21]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [22]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.0948, mae=6.3137, rmse=16.4507
